In [1]:
import spacy
import csv
import random

In [2]:
from typing import Optional, Union, Tuple, List, Callable, Dict
from tqdm.notebook import tqdm
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
import torch.nn.functional as nnf
import numpy as np
import abc
import ptp_utils
import seq_aligner
import shutil
from torch.optim.adam import Adam
from PIL import Image
import os

In [3]:
nlp = spacy.load("en_core_web_sm")
colors=["red","yellow","green","purple","white","brown","blue","orange","pink","grey"]
cloths=["coat","jacket","shirt","dress","blouse","top","hoodie","suit","clothes","cloth",
        "pants","shorts","sweatshirt","skirt","robe","romper","shirts","sweater",
        "clothing","tee-shirt"]

In [7]:
id = 4
csv_reader = csv.reader(open("/data2/liuzhibin/STM/processed_images_cuhk_all/processed_images_cuhk_all_"+str(id)+".csv"))
count=0
prompts=[]
paths=[]
phases=[]
rows=[]
for index,row in enumerate(csv_reader):
    prompt,path=row
    path = os.path.join('/data2/liuzhibin/STM',path)
    '''
    prompt = prompt.replace('[\'','"')
    prompt = prompt.replace('\']','"')
    prompt = prompt.replace('"""','"')
    prompt = prompt.replace('""','"')
    '''
    #prompt = '"'+prompt+'"'
    prompt = prompt.replace('‘', '\'')
    prompt = prompt.replace('’', '\'')
    text = nlp(prompt)
    new_prompt=''
    exist=False
    name = path.strip().split('/')[-1]
    for idx,chunk in enumerate(text.noun_chunks):
        phase = chunk.text
        words = phase.strip().split(' ')
        flag1 = flag2 = False
        for word in words:    
            if word in cloths:
                flag1=True
            if word in colors:
                color = word
                flag2=True
        if flag1 and flag2 and len(prompt.split())<71:
            count+=1
            exist=True
            new_color=color
            while new_color==color:
                new_color=random.choice(colors)
            new_phase=phase.replace(color,new_color)
            new_prompt=prompt.replace(phase,new_phase)
            new_path=os.path.join('./modified_images_cuhk_all/images',name)
            rows.append((prompt,new_prompt,path,new_path,phase,new_phase,new_color))
            break
    
print(count)
with open('./modified_images_cuhk_all/modified_images_'+str(id)+'.csv','w',encoding='utf8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['origin prompt','new prompt','origin path','new path',"origin phase","new phase","new color"])
    writer.writerows(rows)

2223
